In [ ]:
!pip install opendatasets
!pip install pandas

In [ ]:
import opendatasets as od
import pandas
 
 #{"username":"heylovebts","key":"e1f3c4d75d84ca832a973bf07a5b2a33"}
od.download(
    'https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign')

In [4]:
import os
#Name the directory

#google colab directory
#data_dir = '/content/gtsrb-german-traffic-sign'
#train_path = '/content/gtsrb-german-traffic-sign/Train'
#test_path = '/content/gtsrb-german-traffic-sign/test'

#vscode directory
data_dir = './gtsrb-german-traffic-sign'
train_path = './gtsrb-german-traffic-sign/Train'

In [5]:
#The images are categorized into 44 folders based on their signs
NumCategory = len(os.listdir(train_path))
print(NumCategory)

43


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib

#tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
data_dirCOUNT = pathlib.Path(data_dir)

#25346 images in total
image_count = len(list(data_dirCOUNT.glob('*/*.png')))
print(image_count)

12673


In [7]:
#Define parameters for the loader
img_width = 30
img_height = 30
batch_size = 50
EPOCHS = 10

In [8]:
#Labeling the training data
import cv2
from PIL import Image


images_data = []
labels = []


#Load image data from directory `data_dir`
for i in range(42): #/content/gtsrb-german-traffic-sign/category
    path = train_path+'/{0}/'.format(i)
    print(path)
    images = os.listdir(path)

    for img in images:
    #https://www.geeksforgeeks.org/python-opencv-cv2-imread-method/
    #This method returns an image that is loaded from the specified file
      try:
        image = cv2.imread(path + '/' + img)

        #https://pillow.readthedocs.io/en/stable/reference/Image.html
        #Creates an image memory from an object exporting the array interface (using the buffer protocol).
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((30, 30))
        images_data.append(np.array(resize_image))
        labels.append(i)
      except:
        print("Error in " + img)




./gtsrb-german-traffic-sign/Train/0/
./gtsrb-german-traffic-sign/Train/1/
./gtsrb-german-traffic-sign/Train/2/
./gtsrb-german-traffic-sign/Train/3/
./gtsrb-german-traffic-sign/Train/4/
./gtsrb-german-traffic-sign/Train/5/
./gtsrb-german-traffic-sign/Train/6/
./gtsrb-german-traffic-sign/Train/7/
./gtsrb-german-traffic-sign/Train/8/
./gtsrb-german-traffic-sign/Train/9/
./gtsrb-german-traffic-sign/Train/10/
./gtsrb-german-traffic-sign/Train/11/
./gtsrb-german-traffic-sign/Train/12/
./gtsrb-german-traffic-sign/Train/13/
./gtsrb-german-traffic-sign/Train/14/
./gtsrb-german-traffic-sign/Train/15/
./gtsrb-german-traffic-sign/Train/16/
./gtsrb-german-traffic-sign/Train/17/
./gtsrb-german-traffic-sign/Train/18/
./gtsrb-german-traffic-sign/Train/19/
./gtsrb-german-traffic-sign/Train/20/
./gtsrb-german-traffic-sign/Train/21/
./gtsrb-german-traffic-sign/Train/22/
./gtsrb-german-traffic-sign/Train/23/
./gtsrb-german-traffic-sign/Train/24/
./gtsrb-german-traffic-sign/Train/25/
./gtsrb-german-traffic

In [18]:
# Reform the lists to numpy arrays
images = np.array(images_data)
labels = np.array(labels)

print(images.shape, labels.shape)

(38969, 30, 30, 3) (38969,)


In [9]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [10]:
#Converting the labels into one hot encoding
from keras.utils import to_categorical

y_train = to_categorical(y_train, NumCategory)
y_test = to_categorical(y_test, NumCategory)

print(y_train.shape)
print(y_test.shape)

(31175, 43)
(7794, 43)


In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

# Create a convolutional neural network
model = tf.keras.models.Sequential([

        # 2 Convolutional layer. Learn 32 filters using a 3x3 kernel with a max-pooling layer, using 2x2 pool size
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)
        ),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)
        ),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # Flatten units
        tf.keras.layers.Flatten(),

        # Add a dense hidden layer with 256 units        
        tf.keras.layers.Dense(256, activation="relu"),
        # Add a hidden layer with 40% dropout
        tf.keras.layers.Dropout(0.4),

        # Add an output layer with output units for each of the traffic sign categories
        tf.keras.layers.Dense(NumCategory, activation="softmax")
        ])
    
# Train neural network
model.compile(
        optimizer="adam",
        loss = "categorical_crossentropy",
        metrics=["accuracy"]
    )

In [16]:
# Fit model on training data
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS, validation_data=(X_test, y_test))

Epoch 1/10
624/624 [==============================] - 13s 21ms/step - loss: 2.3653 - accuracy: 0.4985 - val_loss: 0.6436 - val_accuracy: 0.8340
Epoch 2/10
624/624 [==============================] - 14s 22ms/step - loss: 0.5618 - accuracy: 0.8377 - val_loss: 0.2591 - val_accuracy: 0.9301
Epoch 3/10
624/624 [==============================] - 13s 21ms/step - loss: 0.3402 - accuracy: 0.9016 - val_loss: 0.1963 - val_accuracy: 0.9506
Epoch 4/10
624/624 [==============================] - 13s 22ms/step - loss: 0.2691 - accuracy: 0.9254 - val_loss: 0.1532 - val_accuracy: 0.9619
Epoch 5/10
624/624 [==============================] - 13s 21ms/step - loss: 0.2027 - accuracy: 0.9425 - val_loss: 0.1564 - val_accuracy: 0.9610
Epoch 6/10
624/624 [==============================] - 13s 21ms/step - loss: 0.1905 - accuracy: 0.9475 - val_loss: 0.1631 - val_accuracy: 0.9613
Epoch 7/10
624/624 [==============================] - 14s 22ms/step - loss: 0.1683 - accuracy: 0.9540 - val_loss: 0.1460 - val_accuracy:

In [17]:
# Evaluate neural network performance
model.evaluate(X_test,  y_test, verbose=2)

244/244 - 1s - loss: 0.1419 - accuracy: 0.9752 - 928ms/epoch - 4ms/step


[0.141885906457901, 0.9752373695373535]